In [14]:
import pandas as pd
import numpy as np
import warnings
from sklearn.cluster import KMeans
from collections import defaultdict
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

class SignalGenerator:
    def __init__(self, k_user=30, k_item=20, min_support=0.3, 
                 min_confidence=0.5, rating_percentile_threshold=0.8,
                 random_state=42):
        self.k_user = k_user
        self.k_item = k_item
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.rating_percentile_threshold = rating_percentile_threshold
        self.random_state = random_state
        
        self.user_labels_ = None
        self.item_labels_ = None
        self.transaction_list_ = None
        self.rules_ = None
        self.frequent_itemsets_ = None # <--- MODIFIED: เพิ่มตัวแปรสำหรับเก็บ Itemsets
        self.num_users_ = None
        self.num_items_ = None

    def _run_clustering(self, Y: np.ndarray):
        user_labels = KMeans(n_clusters=self.k_user, random_state=self.random_state, n_init=10).fit_predict(Y)
        item_labels = KMeans(n_clusters=self.k_item, random_state=self.random_state, n_init=10).fit_predict(Y.T)
        return user_labels, item_labels

    def _binarize_ratings(self, Y: np.ndarray) -> np.ndarray:
        Y_masked = np.where(Y > 0, Y, np.nan)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            min_ratings = np.nanmin(Y_masked, axis=1, keepdims=True)
            max_ratings = np.nanmax(Y_masked, axis=1, keepdims=True)
        min_ratings = np.nan_to_num(min_ratings, nan=0.0)
        max_ratings = np.nan_to_num(max_ratings, nan=1.0)
        max_ratings[max_ratings == min_ratings] = min_ratings[max_ratings == min_ratings] + 1.0 
        thresholds_per_user = min_ratings + (max_ratings - min_ratings) * self.rating_percentile_threshold
        return (Y >= thresholds_per_user)

    def _create_transaction_list(self, transactions_binarized: np.ndarray):
        transaction_list = []
        for u_idx in range(self.num_users_):
            trans = {f'user_group_{self.user_labels_[u_idx]}'}
            liked_items = np.where(transactions_binarized[u_idx])[0] 
            if len(liked_items) > 0:
                for i_idx in liked_items:
                    trans.add(f'item_{i_idx}')
                    trans.add(f'item_group_{self.item_labels_[i_idx]}')
                transaction_list.append(list(trans))
        return transaction_list

    def _find_association_rules(self):
        if not self.transaction_list_: return pd.DataFrame()
        te = TransactionEncoder()
        te_ary = te.fit(self.transaction_list_).transform(self.transaction_list_)
        df = pd.DataFrame(te_ary, columns=te.columns_)
        
        # หา Frequent Itemsets
        frequent_itemsets = fpgrowth(df, min_support=self.min_support, use_colnames=True)
        
        # <--- MODIFIED: เก็บค่า Itemsets ไว้ใน Class attribute
        self.frequent_itemsets_ = frequent_itemsets 
        
        if frequent_itemsets.empty: return pd.DataFrame()
        
        # สร้าง Rules ต่อ
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=self.min_confidence)
        return rules

    def _filter_rules(self):
        if self.rules_.empty: return self.rules_
        filtered_rules = self.rules_[~self.rules_['consequents'].apply(lambda x: any(str(i).startswith('user_group_') for i in x))]
        return filtered_rules

    def _build_signal_matrix(self, Y: np.ndarray):
        matrix_signal = np.zeros_like(Y, dtype=float)
        item_map = {f'item_{i}': i for i in range(self.num_items_)}
        item_group_map = {f'item_group_{i}': i for i in range(self.k_item)}
        item_to_users = defaultdict(set)
        for user_idx, history in enumerate(self.transaction_list_):
            for item in history:
                item_to_users[item].add(user_idx)
        for antecedents_set, group_df in self.rules_.groupby('antecedents'):
            antecedents = list(antecedents_set) 
            if not antecedents: continue
            try:
                matching_users = item_to_users.get(antecedents[0], set()).copy()
                for item in antecedents[1:]:
                    matching_users.intersection_update(item_to_users.get(item, set()))
            except Exception as e: continue
            if not matching_users: continue
            target_user_indices = list(matching_users)
            for _, rule in group_df.iterrows():
                for consequent in rule['consequents']:
                    if consequent in item_map:
                        col_idx = item_map[consequent] 
                        matrix_signal[target_user_indices, col_idx] += rule['confidence']
                    elif consequent in item_group_map:
                        target_items = np.where(self.item_labels_ == item_group_map[consequent])[0]
                        if target_items.size == 0: continue
                        rows, cols = np.meshgrid(target_user_indices, target_items, indexing='ij')
                        matrix_signal[rows, cols] += rule['confidence']
        return matrix_signal

# --- Demo Execution Code (Modified to show Itemsets) ---

def run_debug_steps():
    # 0. Setup Data & Model
    Y = np.array([[1, 5, 0],
                  [1, 0, 2],
                  [0, 5, 3]])
    
    # ลด min_support ลงเล็กน้อยเพื่อให้เห็น itemsets มากขึ้นในข้อมูลตัวอย่าง
    model = SignalGenerator(k_user=3, k_item=2, 
                            min_support=0.3, # <--- ปรับลดเพื่อให้เห็นตัวอย่างชัดขึ้น
                            min_confidence=0.5, 
                            rating_percentile_threshold=0.8, random_state=42)
    
    print("--- INPUT DATA (Y) ---")
    print(Y)
    print("-" * 30)

    model.num_users_, model.num_items_ = Y.shape

    # Step 1: Clustering
    model.user_labels_, model.item_labels_ = model._run_clustering(Y)

    # Step 2: Binarization
    binarized_Y = model._binarize_ratings(Y)

    # Step 3: Transaction List
    model.transaction_list_ = model._create_transaction_list(binarized_Y)
    print("\n[STEP 3] Transactions:")
    for i, trans in enumerate(model.transaction_list_):
        print(f"User {i}: {trans}")

    # Step 4: Association Rules (With Itemsets View)
    print("\n[STEP 4] FP-Growth & Generate Rules")
    model.rules_ = model._find_association_rules()
    
    # <--- MODIFIED: แสดง Frequent Itemsets (Support ราย Subset)
    print("\n>>> FREQUENT ITEMSETS (Support ราย Subset) <<<")
    if model.frequent_itemsets_ is not None and not model.frequent_itemsets_.empty:
        # เรียงลำดับตาม Support จากมากไปน้อยเพื่อให้อ่านง่าย
        print(model.frequent_itemsets_.sort_values(by='support', ascending=False))
    else:
        print("No frequent itemsets found.")
    print("-" * 30 + "\n")

    if not model.rules_.empty:
        print("Generated Rules:")
        print(model.rules_[['antecedents', 'consequents', 'support', 'confidence']])
    else:
        print("No rules found.")

# Execute
run_debug_steps()

--- INPUT DATA (Y) ---
[[1 5 0]
 [1 0 2]
 [0 5 3]]
------------------------------

[STEP 3] Transactions:
User 0: ['user_group_2', 'item_1', 'item_group_0']
User 1: ['user_group_0', 'item_group_1', 'item_2']
User 2: ['user_group_1', 'item_1', 'item_group_0']

[STEP 4] FP-Growth & Generate Rules

>>> FREQUENT ITEMSETS (Support ราย Subset) <<<
    support                              itemsets
0   +0.6667                        (item_group_0)
7   +0.6667                (item_1, item_group_0)
1   +0.6667                              (item_1)
10  +0.3333  (user_group_2, item_1, item_group_0)
16  +0.3333          (user_group_1, item_group_0)
15  +0.3333                (user_group_1, item_1)
14  +0.3333  (user_group_0, item_group_1, item_2)
13  +0.3333                (user_group_0, item_2)
12  +0.3333                (item_group_1, item_2)
11  +0.3333          (user_group_0, item_group_1)
9   +0.3333          (user_group_2, item_group_0)
8   +0.3333                (user_group_2, item_1)
6   +0

In [15]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings

warnings.filterwarnings("ignore")

# --- 1. Dataset Class ---
class PrepareDataset(Dataset):
    def __init__(self, Y_tensor, S_tensor):
        self.Y = Y_tensor
        self.S = S_tensor

    def __len__(self):
        return self.Y.shape[0]

    def __getitem__(self, user_idx):
        return user_idx, self.Y[user_idx], self.S[user_idx]

# --- 2. CARMS Model (Enhanced Debugging) ---
class CARMS_MF(torch.nn.Module):
    
    def __init__(self, user_count, item_count, K, learning_rate, lambda_rate, gamma):
        super().__init__()
        torch.manual_seed(42)
        
        self.user_count = user_count
        self.item_count = item_count
        self.K = K
        self.learning_rate = learning_rate
        self.lambda_rate = lambda_rate
        self.gamma = gamma

        # Embeddings
        self.P_embedding = torch.nn.Embedding(self.user_count, self.K) # User Latent
        self.Q_embedding = torch.nn.Embedding(self.item_count, self.K) # Item Latent for Y
        self.R_embedding = torch.nn.Embedding(self.item_count, self.K) # Item Latent for S

        # Init Weights
        nn.init.normal_(self.P_embedding.weight, std=0.01)
        nn.init.normal_(self.Q_embedding.weight, std=0.01)
        nn.init.normal_(self.R_embedding.weight, std=0.01)

        self.device = torch.device("cpu")
        self.loss_fn = nn.MSELoss(reduction='sum')

    def forward(self, user_indices):
        user_factors = self.P_embedding(user_indices)
        item_factors_y = self.Q_embedding.weight
        item_factors_s = self.R_embedding.weight
        
        # Matrix Multiplication
        y_prediction = (user_factors @ item_factors_y.T)
        s_prediction = (user_factors @ item_factors_s.T)
        
        return y_prediction, s_prediction

    def fit(self, Y, S, epochs, batch_size=1):
        self.to(self.device)
        self.train()

        Y_tensor = torch.tensor(Y, dtype=torch.float32)
        S_tensor = torch.tensor(S, dtype=torch.float32)

        dataset = PrepareDataset(Y_tensor, S_tensor)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.lambda_rate
        )
        
        print(f"Start Training: {epochs} Epochs, Batch Size: {batch_size}")
        print("="*60)

        for epoch in range(epochs):
            total_loss = 0
            print(f"\n>>> EPOCH {epoch+1} START <<<")
            
            step = 1
            for user_indices, Y_batch, S_batch in dataloader:
                u_idx = user_indices.item()
                print(f"\n" + "-"*40)
                print(f"[Step {step}] Processing User Index: {u_idx}")

                user_indices = user_indices.to(self.device)
                Y_batch = Y_batch.to(self.device)
                S_batch = S_batch.to(self.device)
                
                # --- A. Show Current Vectors (Before Update) ---
                # ดึงค่า weight ปัจจุบันออกมาดู (ใช้ .detach() เพื่อไม่ให้กระทบ graph)
                curr_P = self.P_embedding.weight.data[u_idx].numpy().round(4)
                # สำหรับ Q และ R ขอแสดงแค่ 2 items แรกพอสังเขป (หรือแสดงหมดถ้า item น้อย)
                curr_Q = self.Q_embedding.weight.data.numpy().round(4) 
                
                print(f" 1. Current Vectors (Before Update):")
                print(f"    - User P[{u_idx}]: {curr_P}")
                print(f"    - Item Q (All): \n{curr_Q}")

                # --- B. Forward & Error Calc ---
                optimizer.zero_grad() # Clear gradients
                y_hat, s_hat = self(user_indices)
                
                # Calculate Raw Error (e_ui = y_ui - y_hat)
                # Note: MSE Loss จะยกกำลังสองค่านี้ แต่ Gradient จะมาจากค่านี้โดยตรง
                raw_error_y = (Y_batch - y_hat).detach().numpy().flatten().round(4)
                
                print(f" 2. Calculation:")
                print(f"    - Target Y: {Y_batch.numpy().flatten()}")
                print(f"    - Pred   Y: {y_hat.detach().numpy().flatten().round(4)}")
                print(f"    - Error (Target - Pred): {raw_error_y}")

                # --- C. Loss Calculation ---
                mask_y = Y_batch > 0
                loss_y = self.loss_fn(y_hat[mask_y], Y_batch[mask_y])
                
                mask_s = S_batch != 0
                loss_s = self.loss_fn(s_hat[mask_s], S_batch[mask_s])
                
                loss = loss_y + (self.gamma * loss_s)
                
                print(f"    - Loss Value: {loss.item():.6f}")

                # --- D. Backward (Compute Gradients) ---
                loss.backward()
                
                # --- E. Show Gradients ---
                # Gradient จะถูกเก็บไว้ใน .grad ของ parameters
                grad_P = self.P_embedding.weight.grad[u_idx].numpy().round(4)
                grad_Q = self.Q_embedding.weight.grad.numpy().round(4)

                print(f" 3. Gradients (Slope computed from Loss):")
                print(f"    - Grad P[{u_idx}]: {grad_P}")
                print(f"    - Grad Q (All): \n{grad_Q}")
                print(f"      (Note: Grad Q will be close to 0 for items not rated by this user)")

                # --- F. Update Weights ---
                # ใช้ Adam Optimizer ปรับค่า Weight
                # หมายเหตุ: ค่าที่เปลี่ยนจะไม่เท่ากับ lr * grad เป๊ะๆ เพราะ Adam มี momentum
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=10.0)
                optimizer.step()
                
                new_P = self.P_embedding.weight.data[u_idx].numpy().round(4)
                print(f" 4. Updated Vectors:")
                print(f"    - New User P[{u_idx}]: {new_P}")
                
                step += 1
                total_loss += loss.item()

            print(f"\n>>> EPOCH {epoch+1} END. Avg Loss: {total_loss/len(dataloader):.4f}")

# --- 3. Execution ---

# Data Setup (เหมือนเดิม)
Y = np.array([[1, 5, 0],
              [1, 0, 2],
              [0, 5, 3]])

S = np.array([[0.0, 0.6931, 0.0],
              [0.0, 0.0, 0.6931],
              [0.0, 0.6931, 0.0]])

# Model Parameters
K = 2 
learning_rate = 0.01 
lambda_rate = 0.001
gamma = 0.5 

model = CARMS_MF(user_count=3, item_count=3, K=K, 
                 learning_rate=learning_rate, lambda_rate=lambda_rate, gamma=gamma)

# Run Fit
model.fit(Y, S, epochs=1, batch_size=1)

Start Training: 1 Epochs, Batch Size: 1

>>> EPOCH 1 START <<<

----------------------------------------
[Step 1] Processing User Index: 0
 1. Current Vectors (Before Update):
    - User P[0]: [-0.0077 -0.0075]
    - Item Q (All): 
[[ 0.0028  0.0006]
 [ 0.0052 -0.0024]
 [-0.0005  0.0053]]
 2. Calculation:
    - Target Y: [1. 5. 0.]
    - Pred   Y: [-0. -0. -0.]
    - Error (Target - Pred): [1. 5. 0.]
    - Loss Value: 26.240517
 3. Gradients (Slope computed from Loss):
    - Grad P[0]: [-0.0588  0.0167]
    - Grad Q (All): 
[[ 0.0153  0.015 ]
 [ 0.0766  0.0751]
 [-0.     -0.    ]]
      (Note: Grad Q will be close to 0 for items not rated by this user)
 4. Updated Vectors:
    - New User P[0]: [ 0.0023 -0.0175]

----------------------------------------
[Step 2] Processing User Index: 1
 1. Current Vectors (Before Update):
    - User P[1]: [ 0.0035 -0.0031]
    - Item Q (All): 
[[-0.0072 -0.0094]
 [-0.0048 -0.0124]
 [ 0.0093 -0.0047]]
 2. Calculation:
    - Target Y: [1. 0. 2.]
    - Pr